In [98]:
import openai
openai.api_key_path = "/Users/sergmiller/.openai2"
import dsp
import dspy

In [85]:
# we need to add some relevant and irrelevant questions to double-check the logic

train = [
    # base questions
    ('Who produced the album that included a re-recording of "Lithium"?', ['Butch Vig']),
    ('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', ['Kevin Greutert']),
    ('The heir to the Du Pont family fortune sponsored what wrestling team?', ['Foxcatcher', 'Team Foxcatcher', 'Foxcatcher Team']),
    ('In what year was the star of To Hell and Back born?', ['1925']),
    ('Which award did the first book of Gary Zukav receive?', ['U.S. National Book Award', 'National Book Award']),
    ('What city was the victim of Joseph Druces working in?', ['Boston, Massachusetts', 'Boston']),
    # extra, with the same entities (persons) mentioned
    ('Who founded Smart Studios in Madison, while still performing drums in Spooner at night and driving a taxi cab during the day?', ['Butch Vig']),
    ('There were two founders of recording studio called "Smart Studios". The first one was Butch Vig. Who was the second founder?', ['Steve Marker']),

    ('What is the first book of Gary Zukav?', ['"The Dancing Wu Li Masters"', "'The Dancing Wu Li Masters'", "The Dancing Wu Li Masters"]),
    ('When did Zukav returned to the United States?', ['1970'])
]

train = [dsp.Example(question=question, answer=None) for question, answer in train]

In [86]:
QUERY_IDX = 0

# context manager is working properly
with dsp.settings.context(vectorizer=dsp.SentenceTransformersVectorizer()):
    knn_func = dsp.knn(train)

    knn_res_train_vec = knn_func(train[QUERY_IDX], 3)
    print(train[QUERY_IDX].question)  # the query itself
    # similar questions
    print('-'*80)
    for i in [train[train.index(i)].question for i in knn_res_train_vec]:
        print(i)

Who produced the album that included a re-recording of "Lithium"?
--------------------------------------------------------------------------------
Who produced the album that included a re-recording of "Lithium"?
Who founded Smart Studios in Madison, while still performing drums in Spooner at night and driving a taxi cab during the day?
There were two founders of recording studio called "Smart Studios". The first one was Butch Vig. Who was the second founder?


In [8]:
dsp.settings.configure(vectorizer=dsp.SentenceTransformersVectorizer())

In [9]:
QUERY_IDX = 4

knn_res_train_vec = knn_func(train[QUERY_IDX], 3)
print(train[QUERY_IDX].question)  # the query itself
# similar questions
print('-'*80)
for i in [train[train.index(i)].question for i in knn_res_train_vec]:
    print(i)

Which award did the first book of Gary Zukav receive?
--------------------------------------------------------------------------------
Which award did the first book of Gary Zukav receive?
What is the first book of Gary Zukav?
When did Zukav returned to the United States?


In [66]:
class RAG(dspy.Module):
    def __init__(self, num_passages=5):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")

    def forward(self, question):
        context = self.retrieve(question).passages
        answer = self.generate_answer(context=context, question=question)
        return answer, context

In [33]:
from sentence_transformers import SentenceTransformer

In [95]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo')  # can be replaced https://github.com/stanfordnlp/dspy/blob/7d578638d070818f319dc892bb662c435d1cc1bd/docs/using_local_models.md#hfmodel
# lm = dspy.HFClientTGI(model="meta-llama/Llama-2-7b-hf", port=8080, url="http://localhost")
# lm = dspy.HFModel(model = 'meta-llama/Llama-2-7b-hf')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)

In [99]:
message = "population of Helsinki in 2011"

In [100]:
lm.basic_request(message)

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x131997a90>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [74]:
turbo.basic_request(message)

<OpenAIObject chat.completion id=chatcmpl-8Jf9DLQ7N2tRFswgVawSzkHbFJcHr at 0x1316be5e0> JSON: {
  "id": "chatcmpl-8Jf9DLQ7N2tRFswgVawSzkHbFJcHr",
  "object": "chat.completion",
  "created": 1699697095,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The population of Helsinki in 2011 was approximately 596,233."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 14,
    "completion_tokens": 15,
    "total_tokens": 29
  }
}

In [75]:
# with dsp.settings.context(vectorizer=dsp.SentenceTransformersVectorizer()):
rag = RAG()  # zero-shot, uncompiled version of RAG
ans, context = rag(message)

In [90]:
ans.answer

'The population of Helsinki in 2011 was 591,306.'

In [77]:
context

['Helsinki (electoral district) | Helsinki is an electoral district represented in the Finnish "Eduskunta" (parliament). It covers the administrative area of the city of Helsinki, with a population of 591,306 (as of 2011 ). Helsinki currently elects 22 members of the "Eduskunta".',
 'Helsinki | Helsinki ( ; ] ; Swedish: "Helsingfors" ; ] ) is the capital and largest city of Finland. It is in the region of Uusimaa, in southern Finland, on the shore of the Gulf of Finland. Helsinki has a population of , an urban population of , and a metropolitan population of over 1.4 million, making it the most populous municipality and urban area in Finland. Helsinki is some 80 km north of Tallinn, Estonia, 400 km east of Stockholm, Sweden, and 390 km west of Saint Petersburg, Russia. Helsinki has close historical connections with these three cities.',
 'Laajasalo | Laajasalo (Swedish: Degerö ) is a group of islands that forms a neighbourhood in southern Helsinki, the capital of Finland. s of 2010 , i